# OmniShelf AI - YOLOv11 Training on Google Colab

**Project**: Retail Shelf Intelligence with YOLOv11

**Student**: Sukriti Sehgal

**Hardware**: Tesla T4 GPU (16GB VRAM, 12GB RAM)

**Estimated Time**: 2-3 hours for 50 epochs

## Step 1: Mount Google Drive

Upload `omnishelf_colab.zip` to your Google Drive first!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 2: Extract Project Files

In [ ]:
import zipfile
import os

# Extract the zip file
!unzip -q /content/drive/MyDrive/omnishelf_colab.zip -d /content/

# Change to project directory
%cd /content/omnishelf_colab

# Verify files
!ls -lh yolo/dataset/grozi120/

## Step 3: Install Dependencies

In [ ]:
!pip install -q ultralytics pandas pyyaml python-dotenv

# Verify GPU
import torch
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 4: Verify Dataset

In [ ]:
import yaml
from pathlib import Path

# Check data.yaml
with open('yolo/dataset/grozi120/data.yaml') as f:
    config = yaml.safe_load(f)
    print(f"✅ Classes: {config['nc']}")
    print(f"✅ Train path: {config['train']}")
    print(f"✅ Val path: {config['val']}")

# Count images
train_dir = Path('yolo/dataset/grozi120/images/train')
val_dir = Path('yolo/dataset/grozi120/images/val')
print(f"✅ Train images: {len(list(train_dir.iterdir()))}")
print(f"✅ Val images: {len(list(val_dir.iterdir()))}")

## Step 5: Start Training 🚀

**Configuration**:
- Epochs: 50
- Batch: 16 (optimized for T4 GPU)
- Image Size: 640×640
- Augmentations: HSV, Rotation, Flips, Mosaic, Mixup
- Checkpoints: Saved every 5 epochs

In [ ]:
!OMNISHELF_YOLO_DEVICE=0 \
OMNISHELF_YOLO_EPOCHS=50 \
OMNISHELF_YOLO_BATCH=16 \
OMNISHELF_YOLO_WORKERS=2 \
OMNISHELF_YOLO_CACHE=ram \
OMNISHELF_YOLO_RUN_NAME=train_colab \
OMNISHELF_YOLO_PLOTS=false \
OMNISHELF_YOLO_SAVE_PERIOD=5 \
python yolo/train_yolo.py

## Step 6: Monitor Progress (Optional)

Run this in a **separate cell** while training to monitor progress

In [ ]:
import pandas as pd
from IPython.display import clear_output
import time

while True:
    try:
        df = pd.read_csv('yolo/runs/train_colab/results.csv')
        clear_output(wait=True)
        
        print("=" * 80)
        print(f"📊 TRAINING PROGRESS: {len(df)}/50 EPOCHS ({len(df)/50*100:.1f}%)")
        print("=" * 80)
        
        latest = df.iloc[-1]
        best_idx = df['metrics/mAP50(B)'].idxmax()
        best = df.iloc[best_idx]
        
        print(f"\n📈 Current Metrics (Epoch {int(latest['epoch'])})")
        print(f"   mAP@50:     {latest['metrics/mAP50(B)']:.4f}")
        print(f"   mAP@50-95:  {latest['metrics/mAP50-95(B)']:.4f}")
        print(f"   Precision:  {latest['metrics/precision(B)']:.4f}")
        print(f"   Recall:     {latest['metrics/recall(B)']:.4f}")
        
        print(f"\n🏆 Best Results")
        print(f"   Best mAP@50: {best['metrics/mAP50(B)']:.4f} (epoch {int(best['epoch'])})")
        
        print(f"\n⏱️  Timing")
        print(f"   Time elapsed: {df['time'].sum() / 3600:.2f} hours")
        print(f"   Estimated remaining: {(50 - len(df)) * df['time'].mean() / 3600:.2f} hours")
        
        print("\nLast 5 epochs:")
        print(df[['epoch', 'metrics/mAP50(B)', 'metrics/precision(B)', 'metrics/recall(B)']].tail(5).to_string(index=False))
        
        time.sleep(30)
    except FileNotFoundError:
        print("⏳ Waiting for training to start...")
        time.sleep(10)
    except KeyboardInterrupt:
        print("\n⚠️  Monitoring stopped")
        break

## Step 7: View Final Results

In [ ]:
import pandas as pd

df = pd.read_csv('yolo/runs/train_colab/results.csv')

print("=" * 80)
print("✅ TRAINING COMPLETE!")
print("=" * 80)

final = df.iloc[-1]
best_idx = df['metrics/mAP50(B)'].idxmax()
best = df.iloc[best_idx]

print(f"\n📈 Final Results (Epoch 50):")
print(f"   mAP@50:     {final['metrics/mAP50(B)']:.4f}")
print(f"   mAP@50-95:  {final['metrics/mAP50-95(B)']:.4f}")
print(f"   Precision:  {final['metrics/precision(B)']:.4f}")
print(f"   Recall:     {final['metrics/recall(B)']:.4f}")

print(f"\n🏆 Best Model (Saved as best.pt):")
print(f"   Best mAP@50: {best['metrics/mAP50(B)']:.4f} (epoch {int(best['epoch'])})")

print(f"\n⏱️  Total Training Time: {df['time'].sum() / 3600:.2f} hours")

print(f"\n💾 Model Location:")
print(f"   yolo/runs/train_colab/weights/best.pt")
print(f"   yolo/runs/train_colab/weights/last.pt")

## Step 8: Download Trained Model

In [ ]:
from google.colab import files

# Download best model
print("Downloading best.pt...")
files.download('yolo/runs/train_colab/weights/best.pt')

# Create results package
print("Creating results package...")
!zip -r colab_training_results.zip yolo/runs/train_colab/ -q
files.download('colab_training_results.zip')

print("✅ Downloads complete!")

## Troubleshooting

### If training crashes or disconnects:

Resume from last checkpoint:

In [ ]:
!OMNISHELF_YOLO_PRETRAINED=yolo/runs/train_colab/weights/last.pt \
OMNISHELF_YOLO_DEVICE=0 \
OMNISHELF_YOLO_EPOCHS=50 \
OMNISHELF_YOLO_BATCH=16 \
OMNISHELF_YOLO_RUN_NAME=train_colab \
python yolo/train_yolo.py